# Nom du notebook

**Définition** : Ce que je veux faire

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [1]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Récupération des logs pour le mois de février et pour plus tard

In [3]:
from query_elastic_search import load_file_if_exists_or_execute_query

QUERY = {
    "query": {
        "bool": { 
          "must": [
            {
              "range": {
               "logfile": {
                  "gte": "2023-02-01"
               }
              }
            }
          ]
        }
      }
}

In [4]:
logs = load_file_if_exists_or_execute_query("", QUERY)

  0%|          | 0/2229301 [00:00<?, ?it/s]

In [9]:
logs

,timestamp,url,idVisit,lastActionDateTime,lastActionTimestamp,referrerName,referrerTypeName,serverTimePretty,logfile,uvi,outil,outilAction,outilEvent,type,timeSpent,ccAction,query,recoSelection,recoType,suggestionPrefix,suggestionSelection,idCc,resultSelection,cc,feedbackType,visited,referrerUrl
0,1675839157,https://code.travail.gouv.fr/outils/preavis-de...,31927608,2023-02-07 21:52:45,1675806765,service-public.fr,Websites,22:52:45,2023-02-07,903338000,Préavis de démission,view_step,infos,outil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1675839165,https://code.travail.gouv.fr/outils/preavis-de...,31927608,2023-02-07 21:52:45,1675806765,service-public.fr,Websites,22:52:45,2023-02-07,903338000,Préavis de démission,view_step,results,outil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1675839161,https://code.travail.gouv.fr/,31927727,2023-02-07 21:52:41,1675806761,,Direct Entry,22:52:41,2023-02-07,-2139361014,NaN,NaN,NaN,home,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1675839158,https://code.travail.gouv.fr/contribution/3043...,31927724,2023-02-07 21:52:38,1675806758,Google,Search Engines,22:52:38,2023-02-07,1521553519,NaN,NaN,NaN,visit_content,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1675837490,https://code.travail.gouv.fr/outils/convention...,31926523,2023-02-07 21:52:38,1675806758,Google,Search Engines,22:52:38,2023-02-07,-1112319603,Trouver sa convention collective,view_step,start,outil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219296,1676248820,https://code.travail.gouv.fr/contribution/843-...,32187860,2023-02-12 15:40:20,1676216420,Google,Search Engines,16:40:20,2023-02-12,190595292,NaN,NaN,NaN,visit_content,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2219297,1676248819,https://code.travail.gouv.fr/contribution/1351...,32187865,2023-02-12 15:40:19,1676216419,Google,Search Engines,16:40:19,2023-02-12,1247349835,NaN,NaN,NaN,visit_content,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2219298,1676248815,https://code.travail.gouv.fr/contribution/16-l...,32187858,2023-02-12 15:40:15,1676216415,Google,Search Engines,16:40:15,2023-02-12,-375508437,NaN,NaN,NaN,visit_content,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2219299,1676248812,https://code.travail.gouv.fr/outils/simulateur...,32187857,2023-02-12 15:40:12,1676216412,Google,Search Engines,16:40:12,2023-02-12,-586859353,Salaire brut/net,view_step,start,outil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
logs[logs["feedbackType"].isin(["positive", "negative", ""])]

,timestamp,url,idVisit,lastActionDateTime,lastActionTimestamp,referrerName,referrerTypeName,serverTimePretty,logfile,uvi,outil,outilAction,outilEvent,type,timeSpent,ccAction,query,recoSelection,recoType,suggestionPrefix,suggestionSelection,idCc,resultSelection,cc,feedbackType,visited,referrerUrl
482,1675821560,https://code.travail.gouv.fr/convention-collec...,31911040,2023-02-07 17:12:26,1675789946,Bing,Search Engines,18:12:26,2023-02-07,-1760579287,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative,/convention-collective/1596-batiment-ouvriers-...,NaN
643,1675822291,https://code.travail.gouv.fr/outils/preavis-de...,31911937,2023-02-07 17:11:31,1675789891,,Direct Entry,18:11:31,2023-02-07,74882069,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,positive,/outils/preavis-demission,NaN
938,1675822040,https://code.travail.gouv.fr/contribution/quel...,31911619,2023-02-07 17:10:25,1675789825,Lilo,Search Engines,18:10:25,2023-02-07,839373212,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative,/contribution/quelle-est-la-duree-du-preavis-e...,NaN
940,1675822050,https://code.travail.gouv.fr/contribution/16-q...,31911619,2023-02-07 17:10:25,1675789825,Lilo,Search Engines,18:10:25,2023-02-07,839373212,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative,/contribution/16-quelle-est-la-duree-du-preavi...,NaN
965,1675822200,https://code.travail.gouv.fr/outils/indemnite-...,31911853,2023-02-07 17:10:21,1675789821,Google,Search Engines,18:10:21,2023-02-07,-1796461168,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,positive,/outils/indemnite-precarite,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2218870,1676249119,https://code.travail.gouv.fr/outils/simulateur...,32188026,2023-02-12 15:45:19,1676216719,Google,Search Engines,16:45:19,2023-02-12,-414047742,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative,/outils/simulateur-embauche,NaN
2218943,1676248182,https://code.travail.gouv.fr/code-du-travail/l...,32187448,2023-02-12 15:44:06,1676216646,Google,Search Engines,16:44:06,2023-02-12,978576358,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative,/code-du-travail/l1234-9,NaN
2218961,1676249008,https://code.travail.gouv.fr/contribution/quan...,32187448,2023-02-12 15:44:06,1676216646,Google,Search Engines,16:44:06,2023-02-12,978576358,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative,/contribution/quand-le-salarie-a-t-il-droit-a-...,NaN
2218963,1676249041,https://code.travail.gouv.fr/convention-collec...,32187448,2023-02-12 15:44:06,1676216646,Google,Search Engines,16:44:06,2023-02-12,978576358,NaN,NaN,NaN,feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,positive,/convention-collective/1483-commerce-de-detail...,NaN


In [20]:
comments = logs[logs["type"] == "feedback_suggestion"][["url", "logfile", "feedbackType"]]

In [21]:
comments.to_csv("comments_fevrier.csv", sep=";", index=False)